In [1]:
# !pip install tensorflow==2.15

In [2]:
from modnet.preprocessing import MODData
from modnet.models import MODNetModel

/root/miniconda3/envs/modnetori/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-21 17:18:24.429652: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-21 17:18:24.518868: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-21 17:18:24.518913: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-21 17:18:24.

In [3]:
data_path = "/root/c/d/lightning-hydra-template/data/mp/raw/mp.2018.6.1.json.zip"
import pandas as pd
raw_data = pd.read_json(data_path)
raw_data.head()

,material_id,structure,graph,formation_energy_per_atom,band_gap,G,K
0,mvc-8151,# generated using pymatgen\ndata_MgSn(GeO3)2\n...,"{'index1': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",-2.166565,2.2274,NaN,NaN
1,mvc-8154,#\#CIF1.1\n###################################...,"{'index1': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",-1.336841,0.0000,NaN,NaN
2,mvc-1148,#\#CIF1.1\n###################################...,"{'index1': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",-2.264626,0.0606,NaN,NaN
3,mvc-14313,#\#CIF1.1\n###################################...,"{'index1': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",-2.301683,1.3002,NaN,NaN
4,mvc-13470,#\#CIF1.1\n###################################...,"{'index1': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1...",-2.935452,0.0546,NaN,NaN


In [4]:
from pymatgen.core import Structure

In [5]:
ori_data = raw_data
raw_data= ori_data[:100]

In [6]:
# Creating MODData
data = MODData(structure_ids=raw_data["material_id"],
               materials = raw_data["structure"].apply(lambda x: Structure.from_str(x, fmt="cif")),
               targets = raw_data[["formation_energy_per_atom","band_gap"]],
               target_names = ["formation_energy_per_atom","band_gap"],
               )


/root/miniconda3/envs/modnetori/lib/python3.9/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


2024-01-21 17:18:53,338 - modnet - INFO - Loaded Matminer2023Featurizer featurizer.


In [7]:
data.get_structure_df()

,structure
id,
mvc-8151,"[[4.48090775 1.87106003 6.39860628] Mg, [2.467..."
mvc-8154,"[[1.45870738 4.2134368 2.43102817] Cr, [4.378..."
mvc-1148,"[[1.93301616 1.93300619 2.00835049] Ba, [ 1.93..."
mvc-14313,"[[ 2.55615694 3.75067424 -0.48044867] Mg, [1...."
mvc-13470,"[[3.30625571 2.30515409 5.95365325] Mg, [0. 0...."
...,...
mvc-12777,"[[3.08995239 1.78398472 0.29257938] Mg, [1.526..."
mvc-16106,"[[3.0547706 1.76367276 6.24822023] Ba, [-3.33..."
mvc-16459,"[[4.25108933 5.02411411 5.36253337] Ca, [1.448..."


In [8]:
data.get_target_df().count()

formation_energy_per_atom    100
band_gap                     100
dtype: int64

In [9]:
import warnings

warnings.filterwarnings('ignore')

In [10]:
data.featurize()

2024-01-21 17:18:53,602 - modnet - INFO - Computing features, this can take time...
2024-01-21 17:18:53,617 - modnet - INFO - Applying composition featurizers...
2024-01-21 17:18:53,652 - modnet - INFO - Applying featurizers (AtomicOrbitals(), AtomicPackingEfficiency(), BandCenter(), ElementFraction(), ElementProperty(data_source=<matminer.utils.data.MagpieData object at 0x7f122e5d6b80>,
                features=['Number', 'MendeleevNumber', 'AtomicWeight',
                          'MeltingT', 'Column', 'Row', 'CovalentRadius',
                          'Electronegativity', 'NsValence', 'NpValence',
                          'NdValence', 'NfValence', 'NValence', 'NsUnfilled',
                          'NpUnfilled', 'NdUnfilled', 'NfUnfilled', 'NUnfilled',
                          'GSvolume_pa', 'GSbandgap', 'GSmagmom',
                          'SpaceGroupNumber'],
                stats=['minimum', 'maximum', 'range', 'mean', 'avg_dev',
                       'mode']), IonProperty(),

MultipleFeaturizer: 100%|██████████| 100/100 [00:02<00:00, 46.66it/s]


2024-01-21 17:20:22,500 - modnet - INFO - Applying oxidation state featurizers...


CompositionToOxidComposition: 100%|██████████| 100/100 [00:01<00:00, 67.38it/s]


2024-01-21 17:20:26,337 - modnet - INFO - Applying featurizers (ElectronegativityDiff(stats=['minimum', 'maximum', 'range', 'mean', 'std_dev']), OxidationStates(stats=['minimum', 'maximum', 'range', 'std_dev'])) to column 'composition_oxid'.


...

2024-01-21 17:20:28,016 - modnet - INFO - Applying structure featurizers...
2024-01-21 17:20:28,033 - modnet - INFO - Applying featurizers (DensityFeatures(), GlobalSymmetryFeatures(), RadialDistributionFunction(), CoulombMatrix(), SineCoulombMatrix(), EwaldEnergy(), BondFractions(), StructuralHeterogeneity(), MaximumPackingEfficiency(), ChemicalOrdering(), XRDPowderPattern(pattern_length=128)) to column 'structure'.


MultipleFeaturizer: 100%|██████████| 100/100 [00:01<00:00, 67.40it/s]


2024-01-21 17:25:03,018 - modnet - INFO - Applying site featurizers...


SiteStatsFingerprint: 100%|██████████| 100/100 [00:01<00:00, 64.54it/s]


2024-01-21 17:34:28,868 - modnet - INFO - Data has successfully been featurized!


## save and load data before feature selection

In [16]:
data.save('out/mp.2018.6.1[:100]_not_selection.moddata')

2024-01-21 17:42:54,295 - modnet - INFO - Data successfully saved as out/mp.2018.6.1[:100]_not_selection.moddata!


In [17]:
data = MODData.load('out/mp.2018.6.1[:100]_not_selection.moddata')

2024-01-21 17:42:56,564 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f13bf525160> object, created with modnet version 0.4.1


In [11]:
from sklearn.model_selection import train_test_split
split = train_test_split(range(100), test_size=0.1, random_state=1234)
train, val = data.split(split)

In [18]:
train.feature_selection(n=50,use_precomputed_cross_nmi=True)

2024-01-21 17:43:06,936 - modnet - INFO - Loading cross NMI from 'Features_cross' file.
2024-01-21 17:43:07,012 - modnet - WARNING - Feature mismatch between precomputed `Features_cross` and `df_featurized`. Missing columns: {'SineCoulombMatrix|sine coulomb matrix eig 51', 'XRDPowderPattern|xrd_64', 'RadialDistributionFunction|rdf [0.50000 - 0.60000]A', 'CoulombMatrix|coulomb matrix eig 74', 'RadialDistributionFunction|rdf [10.60000 - 10.70000]A', 'XRDPowderPattern|xrd_106', 'CoulombMatrix|coulomb matrix eig 42', 'BondFractions|Ca - Re bond frac.', 'XRDPowderPattern|xrd_76', 'BondFractions|W - W bond frac.', 'CoulombMatrix|coulomb matrix eig 84', 'BondFractions|Ca - Mo bond frac.', 'RadialDistributionFunction|rdf [12.40000 - 12.50000]A', 'CoulombMatrix|coulomb matrix eig 43', 'RadialDistributionFunction|rdf [19.60000 - 19.70000]A', 'BondFractions|Pr - Pr bond frac.', 'RadialDistributionFunction|rdf [10.00000 - 10.10000]A', 'BondFractions|Ca - Sn bond frac.', 'BondFractions|Ge - Ge bond

## define model 

In [19]:
model = MODNetModel([[["formation_energy_per_atom","band_gap"]]],
                    weights={"formation_energy_per_atom": 0.7, "band_gap": 0.3},
                    num_neurons = [[256], [128], [16], [16]],
                    n_feat = 50,
                    act =  "elu"
                   )

2024-01-21 17:44:54.813985: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-21 17:44:55.033580: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [20]:
model.fit(train,
          val_fraction = 0.1,
          lr = 0.0002,
          batch_size = 64,
          loss = 'mae',
          epochs = 100,
          verbose = 1,
         )

epoch 0: loss: 1.435, val_loss:1.300 val_mae:1.858
epoch 1: loss: 1.382, val_loss:1.295 val_mae:1.849
epoch 2: loss: 1.343, val_loss:1.289 val_mae:1.841
epoch 3: loss: 1.316, val_loss:1.286 val_mae:1.837
epoch 4: loss: 1.305, val_loss:1.283 val_mae:1.833
epoch 5: loss: 1.292, val_loss:1.282 val_mae:1.831
epoch 6: loss: 1.277, val_loss:1.281 val_mae:1.830
epoch 7: loss: 1.268, val_loss:1.280 val_mae:1.829
epoch 8: loss: 1.269, val_loss:1.279 val_mae:1.827
epoch 9: loss: 1.262, val_loss:1.278 val_mae:1.825
epoch 10: loss: 1.265, val_loss:1.278 val_mae:1.825
epoch 11: loss: 1.259, val_loss:1.278 val_mae:1.826
epoch 12: loss: 1.256, val_loss:1.279 val_mae:1.827
epoch 13: loss: 1.259, val_loss:1.280 val_mae:1.828
epoch 14: loss: 1.259, val_loss:1.281 val_mae:1.830
epoch 15: loss: 1.256, val_loss:1.284 val_mae:1.834
epoch 16: loss: 1.263, val_loss:1.286 val_mae:1.837
epoch 17: loss: 1.257, val_loss:1.286 val_mae:1.838
epoch 18: loss: 1.250, val_loss:1.284 val_mae:1.835
epoch 19: loss: 1.257,

## val data prediction

In [21]:
pred = model.predict(val)
pred.head()

1/1 [==============================] - 0s 220ms/step


,formation_energy_per_atom,band_gap
mvc-14314,-0.467890,-0.679508
mvc-6226,-0.504165,-0.293106
mvc-12765,-1.146019,-0.412086
mvc-4886,-0.329115,-0.686138
mvc-16458,-0.257114,-0.121710


In [22]:
import numpy as np
mae_test = np.absolute(pred.values-val.df_targets.values).mean()
print(f'mae: {mae_test}')
# 

mae: 1.5296545609377399


In [ ]:
import numpy as np
mae_test = np.absolute(pred.values-val.df_targets.values).mean()
print(f'mae: {mae_test}')

mae: 1.3677090584777367


## test data prediction

In [23]:
data_to_predict = MODData(structure_ids=ori_data["material_id"],
               materials = ori_data["structure"].apply(lambda x: Structure.from_str(x, fmt="cif"))[100:150],
               targets = ori_data[["formation_energy_per_atom","band_gap"]][100:150],
               target_names = ["formation_energy_per_atom","band_gap"][100:150],
               )

KeyError: 0

In [ ]:
data_to_predict.featurize()
df_predictions = model.predict(data_to_predict)

In [ ]:
from modnet.models import MODNetModel
model_pretrained = MODNetModel.load()

In [ ]:
from modnet.preprocessing import MODData
data_MP = MODData.load("data/MP_2018.6")
model.predict(data_MP)